# 03. 복원력 (Resiliency)

## 학습 목표

이 노트북에서는 Istio의 **복원력(Resiliency)** 기능을 학습합니다.

**테스트 시나리오:**
1. Timeout - 응답 지연 시 자동 타임아웃
2. Retry - 실패 시 자동 재시도
3. Fault Injection - 장애 시뮬레이션
4. Circuit Breaker - 연쇄 장애 방지

**학습 포인트:**
- 코드 수정 없이 장애 대응 패턴 적용
- 마이크로서비스 장애 격리
- 카오스 엔지니어링 기초

---

## 사전 조건 체크

In [ ]:
# 환경 확인
!kubectl get pods -n istio-demo | grep -E "httpbin|sleep"

---

## 개념 설명: 복원력 패턴

마이크로서비스에서 장애는 **불가피**합니다. 중요한 것은 **우아하게 대처**하는 것입니다.

### 주요 복원력 패턴

| 패턴 | 목적 | 동작 |
|------|------|------|
| Timeout | 무한 대기 방지 | 지정 시간 후 요청 실패 처리 |
| Retry | 일시적 장애 극복 | 실패 시 자동 재시도 |
| Circuit Breaker | 연쇄 장애 방지 | 장애 서비스 요청 차단 |
| Fault Injection | 장애 테스트 | 인위적 장애 주입으로 테스트 |

### Istio의 장점

**전통적 방식:**
```java
// 코드에 직접 구현 필요
@CircuitBreaker(maxRetries=3)
@Timeout(value=2, unit=TimeUnit.SECONDS)
public Response callService() { ... }
```

**Istio 방식:**
```yaml
# YAML 설정만으로 적용
spec:
  timeout: 2s
  retries:
    attempts: 3
```

---

## 시나리오 1: Timeout - 응답 지연 대응

### 시나리오
- `httpbin`의 `/delay/5` 엔드포인트는 5초 지연 후 응답
- VirtualService로 2초 timeout 설정
- 2초 초과 시 504 Gateway Timeout 반환

### 왜 필요한가?
- 느린 서비스가 전체 시스템을 지연시키는 것 방지
- 리소스(스레드, 연결) 고갈 방지
- 빠른 실패로 사용자 경험 개선

In [ ]:
# Timeout VirtualService 적용
!kubectl apply -f ../02-resiliency/virtual-service-timeout.yaml

In [ ]:
# 테스트 1: /get 요청 (빠른 응답 → 성공)
print("테스트 1: /get 요청 (빠른 응답 → 성공 예상)")
!kubectl exec -n istio-demo deploy/sleep -c sleep -- curl -s -o /dev/null -w "HTTP %{http_code} (응답시간: %{time_total}초)\n" http://httpbin:8000/get

In [ ]:
# 테스트 2: /delay/5 요청 (5초 지연 → 2초에 timeout)
print("테스트 2: /delay/5 요청 (5초 지연 → 2초에 timeout 예상)")
!kubectl exec -n istio-demo deploy/sleep -c sleep -- curl -s -o /dev/null -w "HTTP %{http_code} (응답시간: %{time_total}초)\n" http://httpbin:8000/delay/5

**결과 해석:**
- `/get`: HTTP 200, 응답 시간 < 1초
- `/delay/5`: HTTP 504, 응답 시간 ≈ 2초 (5초 대기 없이 빠르게 실패)

In [ ]:
# Timeout VirtualService 삭제
!kubectl delete -f ../02-resiliency/virtual-service-timeout.yaml --ignore-not-found

---

## 시나리오 2: Retry - 자동 재시도

### 시나리오
- `httpbin`의 `/status/503`은 항상 503 에러 반환
- VirtualService로 3회 재시도 설정
- Envoy가 자동으로 재시도 후 최종 결과 반환

### 왜 필요한가?
- 일시적 네트워크 문제 극복
- 서비스 재시작 중 요청 손실 방지
- 코드 수정 없이 안정성 향상

In [ ]:
# Retry VirtualService 적용
!kubectl apply -f ../02-resiliency/virtual-service-retry.yaml

In [ ]:
# /status/503 요청 (3회 재시도 후 최종 503)
print("/status/503 요청 (Envoy가 3회 재시도 후 최종 503 반환)")
!kubectl exec -n istio-demo deploy/sleep -c sleep -- curl -s -o /dev/null -w "HTTP %{http_code}\n" http://httpbin:8000/status/503

In [ ]:
# Envoy 로그에서 재시도 확인
print("Envoy 프록시 로그 (재시도 흔적 확인):")
!kubectl logs -n istio-demo deploy/sleep -c istio-proxy --tail=10 | grep -E "503|upstream" | head -5

In [ ]:
# Retry VirtualService 삭제
!kubectl delete -f ../02-resiliency/virtual-service-retry.yaml --ignore-not-found

---

## 시나리오 3: Fault Injection - 장애 시뮬레이션

### 시나리오
- 50% 요청에 2초 지연 주입
- 10% 요청에 500 에러 주입
- **카오스 엔지니어링**: 실제 장애 상황을 시뮬레이션

### 왜 필요한가?
- 장애 대응 코드 검증
- 시스템 복원력 테스트
- 프로덕션 배포 전 안정성 확인

In [ ]:
# Fault Injection VirtualService 적용
!kubectl apply -f ../02-resiliency/virtual-service-fault-injection.yaml

In [ ]:
# 10회 요청 테스트
import time

success = 0
delayed = 0
error = 0

print("10회 요청 테스트 (일부 지연, 일부 500 에러 예상)")
for i in range(10):
    start = time.time()
    result = !kubectl exec -n istio-demo deploy/sleep -c sleep -- curl -s -o /dev/null -w "%{{http_code}}" http://httpbin:8000/get 2>/dev/null
    elapsed = time.time() - start
    code = result[0] if result else "000"
    
    if code == "500":
        error += 1
        print("X", end="", flush=True)  # 에러
    elif elapsed > 2:
        delayed += 1
        print("~", end="", flush=True)  # 지연
    else:
        success += 1
        print("O", end="", flush=True)  # 성공

print(f"\n\n결과: 성공={success}, 지연={delayed}, 에러={error}")
print("O=정상, ~=지연(2초+), X=500에러")

In [ ]:
# Fault Injection VirtualService 삭제
!kubectl delete -f ../02-resiliency/virtual-service-fault-injection.yaml --ignore-not-found

---

## 시나리오 4: Circuit Breaker - 연쇄 장애 방지

### 시나리오
- 동시 연결 수 제한 (maxConnections: 1)
- 대기 요청 수 제한 (http1MaxPendingRequests: 1)
- 초과 요청은 503으로 즉시 거부 → 장애 격리

### 왜 필요한가?
- 느린 서비스로 인한 연쇄 장애 방지
- 리소스 고갈 방지
- 빠른 실패로 시스템 보호

In [ ]:
# Circuit Breaker DestinationRule 적용
!kubectl apply -f ../02-resiliency/destination-rule-circuit-breaker.yaml

In [ ]:
# 동시 5개 요청 테스트 (일부 503 예상)
print("동시 5개 요청 테스트 (연결 제한으로 일부 503 예상)")
print("각 요청은 /delay/1 (1초 지연)")
print()

# 백그라운드로 5개 요청 실행
!for i in 1 2 3 4 5; do kubectl exec -n istio-demo deploy/sleep -c sleep -- curl -s -o /dev/null -w "요청 $i: HTTP %{http_code}\n" http://httpbin:8000/delay/1 & done; wait

**결과 해석:**
- 일부 요청은 HTTP 200 (연결 성공)
- 나머지 요청은 HTTP 503 (연결 제한으로 거부)
- Circuit Breaker가 과부하 상황에서 시스템 보호

In [ ]:
# Circuit Breaker DestinationRule 삭제
!kubectl delete -f ../02-resiliency/destination-rule-circuit-breaker.yaml --ignore-not-found
print("정리 완료!")

---

## 복원력 테스트 완료!

### 핵심 정리

| 패턴 | Istio 리소스 | 설정 예시 |
|------|-------------|----------|
| Timeout | VirtualService | `timeout: 2s` |
| Retry | VirtualService | `retries.attempts: 3` |
| Fault Injection | VirtualService | `fault.abort`, `fault.delay` |
| Circuit Breaker | DestinationRule | `outlierDetection`, `connectionPool` |

**Istio 복원력 장점:**
- 코드 수정 없이 적용
- 일관된 정책 적용
- 런타임에 설정 변경 가능

### 다음 단계

**[04-보안.ipynb](./04-보안.ipynb)** - Authorization Policy를 통한 접근 제어를 학습합니다.